### RAG Pipeline - Data Ingestion to Vector DB

In [9]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [13]:
### Data Ingestion - Read all PDF files inside the Directory

def parse_pdf_documents(pdf_directory) :

    all_documents = []
    ## Creates Path Object for the Given Directory 
    pdf_dir = Path(pdf_directory)

    ## pdf_files contains list of PDF paths.
    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} files inside the Directory")

    ## So pdf_file is Path Object
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            ## str(pdf_file) --> Converts the Path object to a normal string path
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['author'] = 'Srini'

            all_documents.extend(documents)
            print(f"\n Loaded {len(documents)} Pages")    

        except Exception as e:
            print(f"\n Error: {e}")

    print(f"\nTotal Documents loaded: {len(all_documents)}") 
    return all_documents       


all_pdf_documents = parse_pdf_documents("../data")

Found 2 files inside the Directory

Processing: Design Patterns.pdf

 Loaded 15 Pages

Processing: IOB CREDIT CARD.pdf

 Loaded 1 Pages

Total Documents loaded: 16


In [14]:
all_pdf_documents

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-11-04T17:26:34+00:00', 'author': 'Srini', 'moddate': '2025-11-04T17:26:35+00:00', 'source': '..\\data\\pdf\\Design Patterns.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1', 'source_file': 'Design Patterns.pdf'}, page_content='DESIGN PATTERN \nWhat is a Design Pattern? \nA Design Pattern is a proven solution to a common software design problem. It provides a structured way to \nwrite code that is maintainable, reusable, and scalable. \nThink of it as a blueprint or a best practice that developers follow to solve recurring design issues efficiently. \nWhy Do We Need Design Patterns? \n1. Improves Code Reusability – Common solutions can be applied across different projects. \n2. Enhances Maintainability – Code becomes structured and easy to understand. \n3. Reduces Code Duplication – Encourages reusing existing patterns instead of reinventing solutions. \n4. Supports Scalabil

In [15]:
## Text Spillter

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    ## chunk_overlap defines how many characters from the end of one chunk are repeated at the 
    # beginning of the next chunk.
    ## length_function tells the splitter how to measure the length of your chunks.
    text_spillter = RecursiveCharacterTextSplitter(
        chunk_size= chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n","\n"," ",""]
    )

    split_docs = text_spillter.split_documents(documents)

    print(f"Total {len(documents)} documents splitted into {len(split_docs)} chunks.")

    if split_docs:
        print(f"\nSample Check:")
        print(f"Content: {split_docs[0].page_content[:200]}")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs

In [16]:
chunks = split_documents(all_pdf_documents)
chunks

Total 16 documents splitted into 28 chunks.

Sample Check:
Content: DESIGN PATTERN 
What is a Design Pattern? 
A Design Pattern is a proven solution to a common software design problem. It provides a structured way to 
write code that is maintainable, reusable, and sc
Metadata: {'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-11-04T17:26:34+00:00', 'author': 'Srini', 'moddate': '2025-11-04T17:26:35+00:00', 'source': '..\\data\\pdf\\Design Patterns.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1', 'source_file': 'Design Patterns.pdf'}


[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-11-04T17:26:34+00:00', 'author': 'Srini', 'moddate': '2025-11-04T17:26:35+00:00', 'source': '..\\data\\pdf\\Design Patterns.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1', 'source_file': 'Design Patterns.pdf'}, page_content='DESIGN PATTERN \nWhat is a Design Pattern? \nA Design Pattern is a proven solution to a common software design problem. It provides a structured way to \nwrite code that is maintainable, reusable, and scalable. \nThink of it as a blueprint or a best practice that developers follow to solve recurring design issues efficiently. \nWhy Do We Need Design Patterns? \n1. Improves Code Reusability – Common solutions can be applied across different projects. \n2. Enhances Maintainability – Code becomes structured and easy to understand. \n3. Reduces Code Duplication – Encourages reusing existing patterns instead of reinventing solutions. \n4. Supports Scalabil

## Embedding and Vector DB

In [7]:
import numpy
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity


c:\Srini\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
## Embeddings

class EmbeddingManager:

    def __init__(self, model_name: str="all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """ Load Sentence Transformer Model"""
        try:
            print(f"Loding Embedding Model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding Dimensions is {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error Loading with Model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> numpy.ndarray:
            if not self.model:
                raise ValueError("Model not loaded")
            
            print(f"Generate embeddings for {len(texts)} texts.")
            embedding = self.model.encode(texts, show_progress_bar= True)
            print(f"Generated embedding shape: {embedding.shape}")
            return embedding

## Initialize embedding manager
embedding_manager = EmbeddingManager()
embedding_manager

Loding Embedding Model: all-MiniLM-L6-v2
Model loaded successfully. Embedding Dimensions is 384


## Vector Store

In [26]:
class VectorStore:

    def __init__(self, collection_name: str="pdf_documents", presist_directory: str="../data/vector_store"):
        self.collection_name = collection_name
        self.presist_directory = presist_directory
        self.client = None
        self.collection = None
        self._initialize_vector()
    
    def _initialize_vector(self):
        """ Initialize chromadb Client and Collection """
        try:
            ## Create Persistent chroma DB client
            os.makedirs(self.presist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.presist_directory)

            ## Get or Create Collection
            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata = {"description":"PDF documents embedding for RAG"}
            )
            print(f"Vector Store Initialized.. Collection: {self.collection_name}")
            print(f"Existing documents in Vector store: {self.collection.count()}")

        except Exception as e:
            print(f"Error Initializing Vector DB: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: numpy.ndarray):
        """ 
        Add docuemnts and their embeddings to the Vector store

        Args:
            documents: List of Langchain documents
            embeddings: Corresponding embeddings for the documents
        """   
        if(len(documents) != len(embeddings)):
            raise ValueError("Length of Documents and Embedding must match..")
        
        print(f"Adding {len(documents)} documents to the vector store")

        ## Prepare data forr ChromaDB
        ids = []
        metadatas = []
        documets_text = []
        embeddings_list = []

        for i, (document, embedding) in enumerate(zip(documents, embeddings)):

            ## Generate Id's
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            ## Create Matadata
            metadata = dict(document.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(document.page_content)
            metadatas.append(metadata)

            ## Document content
            documets_text.append(document.page_content)

            ## Embeddings
            embeddings_list.append(embedding.tolist())

        ##Add to Collection
        try:
            self.collection.add(
                ids=ids,
                metadatas=metadatas,
                embeddings=embeddings_list,
                documents= documets_text
            )    
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vector_store = VectorStore()
vector_store        


Vector Store Initialized.. Collection: pdf_documents
Existing documents in Vector store: 28


In [28]:
## Convert text into embeddings
texts = [doc.page_content for doc in chunks]

## Generate embedding
embeddings = embedding_manager.generate_embeddings(texts)

## Store in Vector DB
vector_store.add_documents(chunks, embeddings)

Generate embeddings for 28 texts.


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.26s/it]

Generated embedding shape: (28, 384)
Adding 28 documents to the vector store
Successfully added 28 documents to vector store
Total documents in collection 84
